In [6]:
#Opencv Version use 3.3., Python 2.7.



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip

In [7]:
#image = mpimg.imread('test_images/solidWhiteRight.jpg')
image = mpimg.imread('test_images/solidWhiteCurve.jpg')
#image = mpimg.imread('test_images/solidYellowCurve2.jpg')
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image) 

('This image is:', <type 'numpy.ndarray'>, 'with dimensions:', (540L, 960L, 3L))


In [8]:
def resizeimage(img):
    return cv2.resize(img, (960 , 540), interpolation = cv2.INTER_AREA)

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 3:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, [vertices], ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    left_slope = []
    right_slope = []
    left_intersept = []
    right_intersept = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            deltay = float(y2-y1)
            deltax = float(x2-x1)
            slope_line = deltay/deltax
            #y = mx+c
            mx = slope_line*x1
            intercept = y1 - mx            
            if slope_line < 0:
                left_slope.append(slope_line)
                left_intersept.append(intercept)
            elif slope_line > 0:
                right_slope.append(slope_line)
                right_intersept.append(intercept)
 
    #print neg_intersept, pos_intersept              
    left_avg_slope = np.mean(left_slope) 
    right_avg_slope = np.mean(right_slope) 
    right_in = np.mean(right_intersept)
    left_in = np.mean(left_intersept)
    top_y = 350
    bot_y = 540
    #(y-c)/m = x
    top_right_x = int(abs((top_y - right_in)/right_avg_slope))
    bot_right_x = int(abs((bot_y - right_in)/right_avg_slope))
    top_left_x = int(abs((top_y - left_in)/left_avg_slope))
    bot_left_x = int(abs((bot_y - left_in)/left_avg_slope))
    #print top_left_x, bot_left_x, neg_avg_slope, top_right_x,  bot_right_x, neg_in, neg_intersept   
    cv2.line(img, (top_left_x, top_y), (bot_left_x, bot_y), color, thickness)
    cv2.line(img, (top_right_x, top_y), (bot_right_x, bot_y), color, thickness)
       
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, alpha, betha, gamma ):
    return cv2.addWeighted(initial_img, alpha, img, betha, gamma)


In [9]:
def process_image(image):
    gray_image = grayscale(image)
    #resize_image = resizeimage(gray_image)
    #intial_image = resizeimage(image)
    canny_image = canny(gray_image, 200, 220)
    blur_image = gaussian_blur(canny_image, 7)
    vertices = np.array([(100, 539), (900, 539), (600, 350), (400, 350)], dtype=np.int32)
    #vertices = np.array([(100, 720), (1200, 720), (800, 400), (200, 400)], dtype=np.int32)
    ROI_image = region_of_interest(blur_image, vertices)
    rho = 1 
    theta = np.pi/180 
    threshold = 70     
    min_line_length = 10 
    max_line_gap = 1 
    hough_image = hough_lines(ROI_image, rho, theta, threshold, min_line_length, max_line_gap)
    weight_image = weighted_img(hough_image, image, 0.8, 1, 0)
    #weight_image = weighted_img(hough_image, intial_image, 0.8, 1, 0)
    return weight_image

In [10]:
#yellow_output = 'test_videos_output/challenge.mp4'
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
#yellow_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
#clip2 = VideoFileClip('test_videos/challenge.mp4')
#clip2 = VideoFileClip('test_videos/solidWhiteRight.mp4')
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
yellow_clip.write_videofile(yellow_output, audio=False)
    

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:12<00:00, 54.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

